In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("./dog_breeds/all/"))

# Any results you write to the current directory are saved as output.

## Initial Imports

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
from glob import glob 

from sklearn.utils import shuffle 
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

## Specify a GPU
import os

Using TensorFlow backend.


In [3]:
base_dir = '../input/'
my_dir = '../input/dog_breeds/'
train_folder = os.path.join(my_dir,'train')
valid_folder = os.path.join(my_dir,'valid')
test_folder = os.path.join(my_dir,'test')

# train_folder = os.path.join(base_dir,'base_dir/train')
# valid_folder = os.path.join(base_dir,'base_dir/valid')
# test_folder = os.path.join(base_dir,'base_dir/test')
val_file, train_file, test_file= [os.path.join("../input/dog-breeds", name) 
                                          for name in os.listdir("../input/dog-breeds/")]

In [4]:
df_train = pd.read_csv(train_file)
df_val = pd.read_csv(val_file)

In [5]:
df_test = pd.read_csv(test_file)
# df_data = df.merge(labels, on = "id")
df_test.head()

,breed_id,breed,id
0,n02091032,Italian_greyhound,n02091032_10352.jpg
1,n02099849,Chesapeake_Bay_retriever,n02099849_1997.jpg
2,n02100583,vizsla,n02100583_12639.jpg
3,n02112137,chow,n02112137_2664.jpg
4,n02105855,Shetland_sheepdog,n02105855_11876.jpg


## Only run for the first time: split training data into training and validation

In [6]:
os.makedirs(os.path.join(my_dir, "all/"))

In [7]:
train_y = df_train['breed']

train_folder = os.path.join(my_dir,'train')
valid_folder = os.path.join(my_dir,'valid')
test_folder = os.path.join(my_dir,'test')
for fold in [train_folder, valid_folder, test_folder]:
    for subf in train_y.unique():
        os.makedirs(os.path.join(fold, subf))

In [8]:
origin_data_dir =  os.path.join(my_dir, "all/")
df_train.set_index('id', inplace=True)
df_val.set_index('id', inplace=True)
df_test.set_index('id', inplace=True)

In [9]:
origin_path = '../input/stanford-dogs-dataset/images/Images/'
for breed in os.listdir(origin_path):
    for file in glob(os.path.join(origin_path,breed,'*.jpg')):
        shutil.copyfile(file, os.path.join(origin_data_dir, file.split('/')[-1:][0]))

In [10]:
for image in df_train.index.values:
    breed = str(df_train.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(train_folder, breed, image)
    shutil.copyfile(src, dst)

In [11]:
for image in df_test.index.values:
    breed = str(df_test.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(test_folder, breed, image)
    shutil.copyfile(src, dst)
for image in df_val.index.values:
    breed = str(df_val.loc[image,'breed']) # get the label for a certain image
    src = os.path.join(origin_data_dir, image)
    dst = os.path.join(valid_folder, breed, image)
    shutil.copyfile(src, dst)

## Image Data Preprocessing

In [12]:
IMAGE_SIZE = 299
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(df_test)

train_batch_size = 32
val_batch_size = 32
test_batch_size = 32
print("Num of train samples: %d" % num_train_samples)
print("Num of validation samples: %d" % num_val_samples)
print("Num of test samples: %d" % num_test_samples)

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)
print("train_steps: %d" % train_steps)
print("val_steps: %d" % val_steps)
print("test_steps: %d" % test_steps)

Num of train samples: 13171
Num of validation samples: 3293
Num of test samples: 4116
train_steps: 412
val_steps: 103
test_steps: 129


In [13]:
from keras.preprocessing.image import ImageDataGenerator
## Noramlize RGB values
datagen = ImageDataGenerator(
    rescale=1./255,
    # preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
    horizontal_flip=True,
    vertical_flip=True)

In [14]:
train_gen = datagen.flow_from_directory(train_folder,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_folder,
                                      target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                      batch_size=val_batch_size,
                                      class_mode='categorical')

test_gen = datagen.flow_from_directory(test_folder,
                                       target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                       batch_size=test_batch_size,
                                       class_mode='categorical',
                                       shuffle=False)

Found 13171 images belonging to 120 classes.
Found 3293 images belonging to 120 classes.
Found 4116 images belonging to 120 classes.


## InceptionResNetV2 Model

In [15]:
## InceptionResNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential, Model
from keras import layers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (IMAGE_SIZE, IMAGE_SIZE)
IN_SHAPE = (*IMG_SIZE, 3)

dropout_dense=0.5

conv_base = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
## Version1
# model.add(GlobalAveragePooling2D())
# model.add(layers.Dense(1024, activation = "relu"))
# model.add(layers.Dense(120, activation = "softmax"))

# #Version 2 
# model.add(GlobalAveragePooling2D())
# model.add(layers.Dense(1024))
# model.add(BatchNormalization())
# model.add(Activation("relu"))
# model.add(layers.Dense(120, activation = "softmax"))


# Version 3
model.add(BatchNormalization())
model.add(Dropout(0.001))
model.add(GlobalMaxPooling2D())
#model.add(layers.Dense(120, activation = "sigmoid"))
model.add(layers.Dense(512, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.001))
model.add(layers.Dense(120, activation = "softmax"))

## Version 4
# model.add(Flatten())
# model.add(Dropout(0.001))
# model.add(layers.Dense(120, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.001))
# model.add(layers.Dense(120, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.001))
# model.add(layers.Dense(120, activation = "softmax"))


# conv_base.summary()
conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])
# model.load_weights('../input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 3s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
## Train inception_resnet model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "inception_resnet_v2_relu_softmax_10.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1,restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                              validation_data=val_gen, validation_steps=val_steps,
                              epochs=20,
                              callbacks=[reducel, checkpoint])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
412/412 [==============================] - 394s 956ms/step - loss: 1.8311 - acc: 0.5758 - val_loss: 1.1436 - val_acc: 0.6985

Epoch 00001: val_acc improved from -inf to 0.69845, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 2/20
412/412 [==============================] - 320s 777ms/step - loss: 0.7959 - acc: 0.7710 - val_loss: 0.9274 - val_acc: 0.7419

Epoch 00002: val_acc improved from 0.69845 to 0.74188, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 3/20
412/412 [==============================] - 320s 778ms/step - loss: 0.5209 - acc: 0.8425 - val_loss: 0.8627 - val_acc: 0.7586

Epoch 00003: val_acc improved from 0.74188 to 0.75858, saving model to inception_resnet_v2_relu_softmax_10.h5
Epoch 4/20
412/412 [==============================] - 320s 777ms/step - loss: 0.3723 - acc: 0.8886 - val_loss: 0.9780 - val_acc: 0.7385


In [17]:
# accuracy metric 1
from keras.metrics import categorical_accuracy
result = model.evaluate_generator(test_gen,steps = test_steps)
print(model.metrics_names)
print(result)

['loss', 'acc']
[0.7974663952001677, 0.8141399416909622]
